### Procesamiento y detección de vacios

In [18]:
from datetime import datetime, timedelta
import os

# Parámetros
expected_interval = 10  # Intervalo esperado en minutos entre imágenes
image_folder = "../data/Img_test/"

# Listar y ordenar imágenes por timestamp
images = []
for file in os.listdir(image_folder):
    if file.endswith(".png"):
        try:
            # Extraer el timestamp del nombre de la imagen
            timestamp_str = file.split("_")[3][1:][:-4]  # Ajustar según el formato real
            timestamp = datetime.strptime(timestamp_str, "%Y%m%d%H%M%S")
            images.append((file, timestamp))
        except (IndexError, ValueError) as e:
            print(f"Advertencia: No se pudo procesar el archivo {file}. Error: {e}")

# Ordenar las imágenes por timestamp
images = sorted(images, key=lambda x: x[1])

# Detectar vacíos en la secuencia
gaps = []
for i in range(len(images) - 1):
    current_time = images[i][1]
    next_time = images[i + 1][1]
    delta = (next_time - current_time).total_seconds() / 60  # Diferencia en minutos
    if delta > expected_interval:
        gaps.append((current_time, next_time))

# Mostrar los vacíos detectados o un mensaje si no hay vacíos
if not gaps:
    print("No se detectaron vacíos en la secuencia de imágenes. Todo está completo.")
else:
    print("Vacíos detectados:")
    gap_info = []
    for idx, (start, end) in enumerate(gaps, start=1):
        missing_count = int((end - start).total_seconds() / 60 / expected_interval) - 1
        gap_info.append((start, end, missing_count))
        print(f"{idx}. Desde {start} hasta {end} ({missing_count} imágenes faltantes)")

    # Preguntar al usuario qué vacío desea llenar
    try:
        selected_index = int(input("\nSeleccione el índice del vacío que desea llenar: "))
        if selected_index < 1 or selected_index > len(gap_info):
            raise ValueError("Índice fuera de rango.")
        selected_gap = gap_info[selected_index - 1]
        start_gap, end_gap, missing_count = selected_gap
        print(f"\nHas seleccionado llenar el rango desde {start_gap} hasta {end_gap}, con {missing_count} imágenes faltantes.")
    except ValueError as e:
        print(f"Error: {e}. Por favor, ingrese un número válido.")


Vacíos detectados:
1. Desde 2023-01-01 12:20:20 hasta 2023-01-01 13:00:20 (3 imágenes faltantes)

Has seleccionado llenar el rango desde 2023-01-01 12:20:20 hasta 2023-01-01 13:00:20, con 3 imágenes faltantes.


In [19]:
# Listar y ordenar imágenes por timestamp
image_list = []
for file in os.listdir(image_folder):
    if file.endswith(".png"):
        # Extraer el timestamp del nombre de la imagen
        timestamp_str = file.split("_")[3][1:][:-4]  # Ajustar según el formato de los nombres
        timestamp = datetime.strptime(timestamp_str, "%Y%m%d%H%M%S")
        image_list.append((file, timestamp))

# Ordenar por timestamp
image_list = sorted(image_list, key=lambda x: x[1])

# Función para obtener las imágenes disponibles antes del vacío
def get_images_before_gap(images, gap_start, num_images=10):
    """
    Recupera las imágenes previas al inicio de un vacío.
    
    Args:
        images (list): Lista de imágenes ordenadas por timestamp.
        gap_start (datetime): Timestamp del inicio del vacío.
        num_images (int): Número de imágenes a recuperar.

    Returns:
        list: Imágenes seleccionadas previas al vacío.
    """
    # Filtrar imágenes con timestamps anteriores al inicio del vacío
    images_before_gap = [img for img in images if img[1] < gap_start]
    
    # Seleccionar las últimas num_images imágenes
    if len(images_before_gap) < num_images:
        raise ValueError(f"No hay suficientes imágenes antes del vacío para seleccionar {num_images}.")
    return images_before_gap[-num_images:]

# Recuperar las imágenes previas al vacío seleccionado
num_images_input = 10  # Ajusta según el modelo (ej. 5, 10, 20)
input_images = get_images_before_gap(image_list, start_gap, num_images=num_images_input)

# Mostrar las imágenes seleccionadas
print(f"Imágenes seleccionadas como entrada ({len(input_images)}):")
for img in input_images:
    print(img[0])


Imágenes seleccionadas como entrada (10):
OR_ABI-L2-ACMF-M6_G16_s20230101104020.png
OR_ABI-L2-ACMF-M6_G16_s20230101105020.png
OR_ABI-L2-ACMF-M6_G16_s20230101110020.png
OR_ABI-L2-ACMF-M6_G16_s20230101111020.png
OR_ABI-L2-ACMF-M6_G16_s20230101112020.png
OR_ABI-L2-ACMF-M6_G16_s20230101113020.png
OR_ABI-L2-ACMF-M6_G16_s20230101114020.png
OR_ABI-L2-ACMF-M6_G16_s20230101115020.png
OR_ABI-L2-ACMF-M6_G16_s20230101120020.png
OR_ABI-L2-ACMF-M6_G16_s20230101121020.png


### Pre procesar imagenes

In [20]:
import cv2
import numpy as np

# Parámetros del preprocesamiento
target_size = (64, 64)  # Redimensionar las imágenes a 64x64

# Función para preprocesar una imagen
def preprocess_image(image_path, target_size):
    """Carga y preprocesa una imagen.
    
    Args:
        image_path (str): Ruta a la imagen.
        target_size (tuple): Tamaño de salida (ancho, alto).
    
    Returns:
        np.array: Imagen preprocesada.
    """
    # Cargar la imagen en formato RGB
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Redimensionar
    image = cv2.resize(image, target_size)
    
    # Normalizar valores de píxeles (0-255 -> 0-1)
    image = image / 255.0
    
    return image

# Cargar y preprocesar las imágenes seleccionadas
preprocessed_images = []
for img_name, _ in input_images:
    img_path = os.path.join(image_folder, img_name)
    preprocessed_images.append(preprocess_image(img_path, target_size))

# Convertir a un array numpy
preprocessed_images = np.array(preprocessed_images)

print(f"Imágenes preprocesadas: {preprocessed_images.shape}")


Imágenes preprocesadas: (10, 64, 64, 3)


In [21]:
# Generar los timestamps faltantes
missing_timestamps = [
    start_gap + timedelta(minutes=expected_interval * i)
    for i in range(1, missing_count + 1)
]

# Verificar los timestamps generados
print(f"Timestamps faltantes: {missing_timestamps}")

# Crear un placeholder para las imágenes faltantes
missing_images = []
for ts in missing_timestamps:
    # Placeholder con el tamaño 64x64 y 3 canales
    missing_images.append(np.zeros((64, 64, 3), dtype=np.float32))

# Convertir a un array numpy
missing_images = np.array(missing_images)

print(f"Imágenes faltantes preparadas: {missing_images.shape}")


Timestamps faltantes: [datetime.datetime(2023, 1, 1, 12, 30, 20), datetime.datetime(2023, 1, 1, 12, 40, 20), datetime.datetime(2023, 1, 1, 12, 50, 20)]
Imágenes faltantes preparadas: (3, 64, 64, 3)


### Modelo

In [22]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, TimeDistributed, ConvLSTM2D, BatchNormalization, Dense, Reshape, Lambda, Flatten
)
from tensorflow.keras.optimizers.legacy import Adam
import tensorflow as tf

# Función para redimensionar imágenes dentro del modelo
def resize_output_to_original(x):
    return tf.image.resize(x, [593, 449], method="bilinear")

# Modelo base VGG16 para extracción de características
def build_vgg16_extractor(input_shape=(64, 64, 3)):
    vgg = VGG16(include_top=False, weights="imagenet", input_shape=input_shape)
    model = Model(inputs=vgg.input, outputs=vgg.output)  # Mantén la salida convolucional
    return model

# Crear modelo VGG16 para extracción de características
vgg16_extractor = build_vgg16_extractor()

# Parámetros del modelo
sequence_length = 60  # Tamaño de la secuencia
input_shape = (sequence_length, 64, 64, 3)  # Secuencia de `sequence_length` imágenes de 64x64x3

# Entrada del modelo
input_layer = Input(shape=input_shape)

# Extracción de características con VGG16
x = TimeDistributed(vgg16_extractor)(input_layer)  # Procesa cada imagen de la secuencia

# ConvLSTM para capturar relaciones temporales
x = ConvLSTM2D(filters=32, kernel_size=(3, 3), padding="same", return_sequences=True)(x)
x = BatchNormalization()(x)
x = ConvLSTM2D(filters=32, kernel_size=(3, 3), padding="same", return_sequences=False)(x)
x = BatchNormalization()(x)

# Decodificación de salida
x = Dense(64 * 64 * 3, activation="relu")(Flatten()(x))  # Reconstruir dimensiones
x = Reshape((64, 64, 3))(x)  # Reconstruir forma original de la imagen

# Redimensionar a tamaño original
x = Lambda(resize_output_to_original)(x)

# Crear modelo completo
model = Model(inputs=input_layer, outputs=x)

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.0001), loss="mse")

# Revisar arquitectura del modelo
model.summary()


Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 60, 64, 64, 3)]   0         
                                                                 
 time_distributed_7 (TimeDi  (None, 60, 2, 2, 512)     14714688  
 stributed)                                                      
                                                                 
 conv_lstm2d_7 (ConvLSTM2D)  (None, 60, 2, 2, 32)      626816    
                                                                 
 batch_normalization_6 (Bat  (None, 60, 2, 2, 32)      128       
 chNormalization)                                                
                                                                 
 conv_lstm2d_8 (ConvLSTM2D)  (None, 2, 2, 32)          73856     
                                                                 
 batch_normalization_7 (Bat  (None, 2, 2, 32)          128 

### Entrenamiento del modelo

In [23]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os
import numpy as np
from datetime import datetime, timedelta
import cv2

# Crear X_train y y_train
sequence_length = 60  # Tamaño más grande para capturar más contexto
image_folder = "../data/Img_test/"
images = []

# Listar y ordenar imágenes por timestamp
for file in os.listdir(image_folder):
    if file.endswith(".png"):
        try:
            timestamp_str = file.split("_")[3][1:][:-4]  # Extraer timestamp del nombre
            timestamp = datetime.strptime(timestamp_str, "%Y%m%d%H%M%S")
            images.append((file, timestamp))
        except (IndexError, ValueError):
            print(f"Archivo ignorado: {file}")

images = sorted(images, key=lambda x: x[1])  # Ordenar por timestamp

# Validar si hay suficientes datos
if len(images) < 2 * sequence_length:
    raise ValueError(f"No hay suficientes imágenes para generar secuencias. Se requieren al menos {2 * sequence_length}, pero solo hay {len(images)} disponibles.")

X_train = []
y_train = []

for i in range(len(images) - 2 * sequence_length):
    current_seq = images[i:i + sequence_length]
    next_seq = images[i + sequence_length:i + 2 * sequence_length]
    # Verificar si las secuencias son consecutivas (permitir un rango de tolerancia)
    time_diff = (next_seq[0][1] - current_seq[-1][1]).total_seconds() / 60
    if 9 <= time_diff <= 11:  # Tolerancia de ±1 minuto
        # Procesar imágenes para entrada y salida
        X = [cv2.resize(cv2.imread(os.path.join(image_folder, img[0])), (64, 64)) for img in current_seq]
        y = [cv2.resize(cv2.imread(os.path.join(image_folder, img[0])), (64, 64)) for img in next_seq]
        X_train.append(X)
        y_train.append(y)

# Validar si se generaron datos
if not X_train or not y_train:
    raise ValueError("No se generaron datos para el entrenamiento. Verifica las condiciones de consecutividad o la cantidad de imágenes disponibles.")

# Convertir a arrays numpy
X_train = np.array(X_train)
y_train = np.array(y_train)

print(f"Nuevo X_train shape: {X_train.shape}")
print(f"Nuevo y_train shape: {y_train.shape}")

# Dividir los datos en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Normalizar los datos
X_train = X_train / 255.0
X_val = X_val / 255.0
y_train = y_train / 255.0
y_val = y_val / 255.0

# Entrenamiento del modelo
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint("optimized_model_vgg16_lstm.h5", save_best_only=True, monitor="val_loss")

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,  # Ajustar según los recursos y el tamaño del dataset
    batch_size=4,  # Reducir si hay problemas de memoria
    verbose=1,
    callbacks=[early_stopping, model_checkpoint]
)

# Guardar el modelo final
model.save("final_optimized_model_vgg16_lstm.h5")
print("Entrenamiento finalizado. Modelo guardado como 'final_optimized_model_vgg16_lstm.h5'")


Nuevo X_train shape: (20, 60, 64, 64, 3)
Nuevo y_train shape: (20, 60, 64, 64, 3)
Epoch 1/20


ValueError: in user code:

    File "/Users/santiagoromero/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/Users/santiagoromero/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/santiagoromero/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/Users/santiagoromero/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/santiagoromero/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/Users/santiagoromero/anaconda3/lib/python3.11/site-pac

### Validacion del modelo

In [ ]:
# Cargar el modelo entrenado
from tensorflow.keras.models import load_model

model_path = "final_model_vgg16_lstm.h5"  # Cambia por la ruta correcta si es necesario
model = load_model(model_path)

# Preprocesar las imágenes seleccionadas para el vacío
X_input = np.expand_dims(preprocessed_images, axis=0)  # Agregar dimensión batch

# Generar las imágenes faltantes
y_pred = model.predict(X_input)

print(f"Imágenes generadas: {y_pred.shape}")


In [ ]:
import os
import cv2

output_folder = "../data/Images_Forecast"
os.makedirs(output_folder, exist_ok=True)

# Guardar las imágenes generadas
for pred_image, timestamp in zip(y_pred[0], missing_timestamps):
    # Desnormalizar la imagen
    pred_image = (pred_image * 255).astype(np.uint8)

    # Redimensionar al tamaño original (593x449)
    pred_image_resized = cv2.resize(pred_image, (449, 593))

    # Crear el nombre del archivo
    output_name = f"OR_ABI-L2-ACMF-M6_G16_s{timestamp.strftime('%Y%m%d%H%M%S')}.png"
    output_path = os.path.join(output_folder, output_name)

    # Guardar la imagen
    cv2.imwrite(output_path, cv2.cvtColor(pred_image_resized, cv2.COLOR_RGB2BGR))
    print(f"Imagen generada guardada en: {output_path}")
